Задание 1

In [18]:
import requests

In [19]:
from bs4 import BeautifulSoup

In [20]:
import pandas as pd

In [21]:
# определяем список хабов, которые нам интересны
KEYWORDS = ['python']

In [22]:
# получаем страницу с самыми свежими постами
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')

In [23]:
# извлекаем посты
posts = soup.find_all('article', class_='post')
posts

[<article class="post post_preview" lang="ru">
 <header class="post__meta">
 <a class="post__user-info user-info" href="https://habr.com/ru/users/x-wao/" title="Автор публикации">
 <span class="default-image_mini default-image_lilac">
 <svg aria-hidden="true" class="icon-svg" height="24" role="img" version="1.1" viewbox="0 0 24 24" width="24"><path d="M21.5 24h-19c-1.379 0-2.5-1.122-2.5-2.5v-19c0-1.379 1.122-2.5 2.5-2.5h19c1.379 0 2.5 1.122 2.5 2.5v19c0 1.379-1.122 2.5-2.5 2.5zm-19-23c-.827 0-1.5.673-1.5 1.5v19c0 .827.673 1.5 1.5 1.5h19c.827 0 1.5-.673 1.5-1.5v-19c0-.827-.673-1.5-1.5-1.5h-19zM15.598 12.385zM19.438 15.417l-.002-.005v-.001c-.875-2.226-2.484-3.054-3.445-3.549l-.273-.143c.029-.497-.025-1.034-.167-1.599l-.128.032.123-.044c-.765-2.152-1.757-2.585-2.632-2.967l-.006-.003-.535-2.121c.357-.065.628-.375.628-.752.001-.423-.342-.765-.765-.765s-.766.342-.766.765c0 .358.248.657.581.74l-.825 1.654-.014-.003-.024-.003c-1.053-.033-1.842.369-2.5.947-.633-.322-1.515-.729-2.158-1.814.107-.

In [24]:
kom_news = pd.DataFrame()
for post in posts:
    post_id = post.parent.attrs.get('id')
   # если идентификатор не найден, это что-то странное, пропускаем
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    hubs = post.find_all('a', class_='hub-link')
    for hub in hubs:
            hub_lower = hub.text.lower()
           # ищем вхождение хотя бы одного желаемого хаба
            if any([hub_lower in desired for desired in KEYWORDS]):
               # пост нам интересен - делаем с ним все что захотим:
               # можно отправить в телеграм уведомление, можно на почту и т.п.
                date_title_element = post.find('span', class_='post__time')
                title_element = post.find('a', class_='post__title_link')
                # так как пост уже нам подошел - дальше нет смысла проверять хабы
                row = {'date': date_title_element.text, 'title': title_element.text, 'ref': title_element.attrs.get('href')}
                kom_news = pd.concat([kom_news, pd.DataFrame([row])])  
kom_news

,date,title,ref
0,сегодня в 11:10,Как же писать эти грёбаные циклы?,https://habr.com/ru/post/543382/
0,сегодня в 10:59,Ускоряем код на Python с помощью Nim,https://habr.com/ru/company/otus/blog/543332/


In [ ]:
Задание 1 часть 2

In [25]:
kom_news = pd.DataFrame()
for post in posts:
    post_id = post.parent.attrs.get('id')
   # если идентификатор не найден, это что-то странное, пропускаем
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    hubs = post.find_all('a', class_='hub-link')
    for hub in hubs:
            hub_lower = hub.text.lower()
           # ищем вхождение хотя бы одного желаемого хаба
            if any([hub_lower in desired for desired in KEYWORDS]):
               # пост нам интересен - делаем с ним все что захотим:
               # можно отправить в телеграм уведомление, можно на почту и т.п.
                date_title_element = post.find('span', class_='post__time')
                title_element = post.find('a', class_='post__title_link')
                hrefs = post.find(class_='post__title_link')['href']
                req = requests.get(hrefs)
                soup = BeautifulSoup(requests.get(hrefs).text, 'html.parser')
                if soup.find('div', class_='post__text post__text_v2'):
                    text = soup.find('div', class_='post__text post__text_v2').text
                else:
                    text = soup.find('div', class_='post__text post__text-html post__text_v1').text
                # так как пост уже нам подошел - дальше нет смысла проверять хабы
                row = {'date': date_title_element.text, 'title': title_element.text, 'ref': hrefs, 'text': text}
                kom_news = pd.concat([kom_news, pd.DataFrame([row])])  
kom_news

,date,title,ref,text
0,сегодня в 11:10,Как же писать эти грёбаные циклы?,https://habr.com/ru/post/543382/,"Когда мои ученики, уже выучив основы синтаксис..."
0,сегодня в 10:59,Ускоряем код на Python с помощью Nim,https://habr.com/ru/company/otus/blog/543332/,"Привет, хабровчане. В преддверии старта курса ..."


Задание 2

In [9]:
import time

In [10]:
import json

In [11]:
import pandas as pd

In [12]:
import requests 

In [13]:
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

In [14]:
emails = ['n_shak@mail.ru', 'n.shakirova@dc11.ru', 'n.shakirova@arenda-mir.ru', '111927@mail.ru', '1276_sch@mail.ru', '1297school@mail.ru', 'anna-alekseeva92@mail.ru']

In [15]:
for email in emails:
  params = {'emailAddresses': [email]}
  res = requests.post(URL, data = json.dumps(params), headers = {
    'Vaar-Version': '0',
    'Vaar-Header-App-Product': 'hackcheck-web-avast'
})
  time.sleep(0.3)
  res.json()
  print(res)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [16]:
res.text

'null'

In [105]:
df = pd.DataFrame(res.json()['breaches'])

TypeError: 'NoneType' object is not subscriptable